Invalid characters in RNA or DNA：4912个文件

Overlapping chains：570个文件，都在invalid里

error：5

其他文件：27

统计：
- overlap/invalid characters
- 多少文件含有RNA
- 总共多少条不同的RNA链
- 链的长度分布
- 各种filter

### 1. Overview
- PDB解析错误：5
- 其他文件：27
- 能正确解析的Protein/DNA/RNA的cif文件：212029

In [1]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import json
with open("/expanse/ceph/projects/itasser/jlspzw/nwentao/pdb_mmcif/mmcif_cache.json", "r") as f:
    mmcif_cache = json.load(f)
len(mmcif_cache)

含有RNA链的cif文件

In [ ]:
has_rna = {}
for k, v in mmcif_cache.items():
    if v["no_chains"]["rna"]:
        has_rna[k] = v["no_chains"]["rna"]
len(has_rna)

: 

### 2. 排除overlap的文件

In [ ]:
with open("/expanse/ceph/projects/itasser/jlspzw/nwentao/pdb_mmcif/mmcif_cache_log.json", "r") as f:
    mmcif_cache_log = json.load(f)

: 

In [ ]:
for k in mmcif_cache_log["overlap"]:
    mmcif_cache.pop(k)
len(mmcif_cache)

: 

含有RNA链的cif文件

In [ ]:
has_rna = []
for k, v in mmcif_cache.items():
    if v["no_chains"]["rna"]:
        has_rna.append(k)
len(has_rna)

: 

RNA单链总数

In [ ]:
rna_chains = {}
for file_id in has_rna:
    for chain_ids, chain in mmcif_cache[file_id]["rna"].items():
        rna_chains[file_id + "_" + chain_ids] = chain
len(rna_chains)

: 

### 过滤

Resolution <= 4.5Å

In [ ]:
filter1 = []
for file_id in has_rna:
    if mmcif_cache[file_id]["header"]["resolution"] <= 4.5:
        filter1.append(file_id)
len(filter1)

: 

整理格式

In [ ]:
rna_chains = {}
for file_id in filter1:
    for chain_ids, chain in mmcif_cache[file_id]["rna"].items():
        rna_chains[file_id + "_" + chain_ids] = chain

: 

将其他字符转化为X

In [ ]:
for k,v in rna_chains.items():
    v = ''.join(['X' if nucleotide not in 'AUCG' else nucleotide for nucleotide in v])
    rna_chains[k] = v

: 

链长统计

In [ ]:
lengths = list(map(lambda x:len(x), rna_chains.values()))

: 

In [ ]:
lengths2 = np.array(lengths)
lengths2 = lengths2[lengths2 <= 500]
sns.histplot(lengths2, bins = 80)
plt.title('Chain Length Distribution (<=500)')

: 

In [ ]:
lengths2 = np.array(lengths)
lengths2 = lengths2[lengths2 > 500]
lengths2[lengths2 > 5000] = 5000
sns.histplot(lengths2, bins = 80)
plt.title('Chain Length Distribution (>500)')

: 

设置链长范围

In [ ]:
minl = 15
maxl = np.inf
valid_length = {}
for k,v in rna_chains.items():
    if len(v) <= maxl and len(v) >= minl:
        valid_length[k] = v
len(valid_length)

: 

Any single nucleotide accounts for more than 90%

In [ ]:
most_prop = []
for k,v in rna_chains.items():
    s = max(v, key=v.count)
    most_prop.append(v.count(s)/len(v))
sns.histplot(most_prop)
plt.title('Maximum proportion of any single nucleotide')

: 

In [ ]:
valid_most = {}
for k,v in rna_chains.items():
    s = max(v, key=v.count)
    if v.count(s)/len(v) < 0.9:
        valid_most[k] = v
len(valid_most)

: 

Unknown nucleotide “X” accounts for more than 50%

In [ ]:
X_prop = []
for k,v in rna_chains.items():
    X_prop.append(v.count("X")/len(v))

: 

In [ ]:
(np.array(X_prop) > 0.05).sum()

: 

In [ ]:
valid_X = {}
for k,v in rna_chains.items():
    if v.count("X")/len(v) < 0.5:
        valid_X[k] = v
len(valid_X)

: 

最后剩下的链

In [ ]:
keys = set(valid_X.keys()) & set(valid_most.keys()) & set(valid_length.keys())
valid_total = {k: rna_chains[k] for k in rna_chains if k in keys}
len(valid_total)

: 

In [ ]:
string = '4v9i_AA'
file_id, chain_ids = string.split("_")
chain_ids = chain_ids.split(",")
term = "Chains " if len(chain_ids) > 1 else "Chain "
file_id.upper() + "|" + term + ", ".join(chain_ids)

'4V9I|Chain AA'

In [1]:
from demofold.model.structure_module import StructureModule

d:\anaconda3\envs\scanpy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = StructureModule(256, 128, 64, 32, 8, 4, 4, 0.02, 4, 1, 2, 6, 10, 1e-7, 1e7)

In [3]:
import torch
evoformer_output_dict = {
    "single": torch.randn(4, 77, 256),
    "pair": torch.randn(4, 77, 77, 128)
}
x = model(evoformer_output_dict)